In [1]:
import xmltodict
import os
import pandas as pd

In [2]:
color_code = {'critical':'#ff4660', #red
              'high':'#f48154',  #orange
             'medium':'#d9ff7f', #yellow
             'low':'#42ff58', #green
             'informational':'#54ba8a'} #blue

In [3]:
with open(os.path.expanduser('~/NorsePi/XML/LastHour.xml'),'r') as fd:
    doc = xmltodict.parse(fd.read())

In [4]:
reform = {(outerKey, innerKey): values for outerKey, innerDict in doc.items() for innerKey, values in innerDict.items()}
a = pd.DataFrame(reform)
reform = a['response']['result']['log']
a = pd.DataFrame(reform)

total = a['logs']['@count']
reform = a['logs']['entry']
df = pd.DataFrame(reform)

df = df[['direction','device_name','time_generated',"src",'srcloc','dst','dstloc','subtype','threatid','severity']]
df['srcname'] = ''
df['srclat'] = ''
df['srclong'] = ''
df['dstname'] = ''
df['dstlat'] = ''
df['dstlong'] = ''
df['src_alpha2'] = ''
df['dst_alpha2'] = ''
df['color_ataque'] = ''

for idx,i in enumerate(df['dstloc']):
    df['dst_alpha2'][idx] = i['@cc']
for idx,i in enumerate(df['srcloc']):
    df['src_alpha2'][idx] = i['@cc']
for idx in range(len(df)):
    print(idx)
    df.iloc[idx]['color_ataque'] = color_code[df.iloc[idx]['severity']]
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

2369
2370
2371
2372
2373
2374
2375
2376
2377
2378
2379
2380
2381
2382
2383
2384
2385
2386
2387
2388
2389
2390
2391
2392
2393
2394
2395
2396
2397
2398
2399
2400
2401
2402
2403
2404
2405
2406
2407
2408
2409
2410
2411
2412
2413
2414
2415
2416
2417
2418
2419
2420
2421
2422
2423
2424
2425
2426
2427
2428
2429
2430
2431
2432
2433
2434
2435
2436
2437
2438
2439
2440
2441
2442
2443
2444
2445
2446
2447
2448
2449
2450
2451
2452
2453
2454
2455
2456
2457
2458
2459
2460
2461
2462
2463
2464
2465
2466
2467
2468
2469
2470
2471
2472
2473
2474
2475
2476
2477
2478
2479
2480
2481
2482
2483
2484
2485
2486
2487
2488
2489
2490
2491
2492
2493
2494
2495
2496
2497
2498
2499
2500
2501
2502
2503
2504
2505
2506
2507
2508
2509
2510
2511
2512
2513
2514
2515
2516
2517
2518
2519
2520
2521
2522
2523
2524
2525
2526
2527
2528
2529
2530
2531
2532
2533
2534
2535
2536
2537
2538
2539
2540
2541
2542
2543
2544
2545
2546
2547
2548
2549
2550
2551
2552
2553
2554
2555
2556
2557
2558
2559
2560
2561
2562
2563
2564
2565
2566
2567
2568


4588
4589
4590
4591
4592
4593
4594
4595
4596
4597
4598
4599
4600
4601
4602
4603
4604
4605
4606
4607
4608
4609
4610
4611
4612
4613
4614
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4630
4631
4632
4633
4634
4635
4636
4637
4638
4639
4640
4641
4642
4643
4644
4645
4646
4647
4648
4649
4650
4651
4652
4653
4654
4655
4656
4657
4658
4659
4660
4661
4662
4663
4664
4665
4666
4667
4668
4669
4670
4671
4672
4673
4674
4675
4676
4677
4678
4679
4680
4681
4682
4683
4684
4685
4686
4687
4688
4689
4690
4691
4692
4693
4694
4695
4696
4697
4698
4699
4700
4701
4702
4703
4704
4705
4706
4707
4708
4709
4710
4711
4712
4713
4714
4715
4716
4717
4718
4719
4720
4721
4722
4723
4724
4725
4726
4727
4728
4729
4730
4731
4732
4733
4734
4735
4736
4737
4738
4739
4740
4741
4742
4743
4744
4745
4746
4747
4748
4749
4750
4751
4752
4753
4754
4755
4756
4757
4758
4759
4760
4761
4762
4763
4764
4765
4766
4767
4768
4769
4770
4771
4772
4773
4774
4775
4776
4777
4778
4779
4780
4781
4782
4783
4784
4785
4786
4787


In [5]:
paises = pd.read_csv(os.path.expanduser('~/NorsePi/CSV/country_centroids_primary.csv'),sep='\t')

campi = pd.read_csv(os.path.expanduser('~/NorsePi/CSV/GPSTec _copy.csv'))

In [6]:
# TODO Ler lista Felipe
campusDevice = pd.read_csv(os.path.expanduser('/home/gabriel/NorsePi/CSV/GPSTec.csv'))

In [19]:
notFound = set()
for idx in range(len(df)):
    # Source
    try:
            # Checa si el origen es un pais
        tmp = paises[paises['FIPS10'] == df.iloc[idx]['src_alpha2']] # Intenta con un campo de CSV
        if len(tmp) == 0: # Intenta con otro campo
            tmp = paises[paises['ISO3136'] == df.iloc[idx]['src_alpha2']]
        if len(tmp) != 0: # Si encontró un pais...
            df.iloc[idx]['srclat'] = tmp['LAT'].values[0]
            df.iloc[idx]['srclong'] = tmp['LONG'].values[0]
            df.iloc[idx]['srcname'] = tmp['SHORT_NAME'].values[0]
        else: # Si no encuentra, la fuente es un campus (Pondremos como MTY el default)
            tmp = campusDevice[campusDevice['Device'] == 'CP-MTY-1']
            notFound.add(idx)
            print('src MTY:',idx)
            df.iloc[idx]['srclat'] = tmp['latitud'].values[0]
            df.iloc[idx]['srclong'] = tmp['longitud'].values[0]
            df.iloc[idx]['srcname'] = tmp['Nombre'].values[0]
    except Exception as e:
        notFound.add(idx)
        print('source:',idx,e)
    # Destiny
    try:
            # Checa si el destino es un pais
        tmp = paises[paises['FIPS10'] == df.iloc[idx]['dst_alpha2']] # Intenta con un campo de CSV
        if len(tmp) == 0: # Intenta con otro campo
            tmp = paises[paises['ISO3136'] == df.iloc[idx]['dst_alpha2']]
        if len(tmp) != 0: # Si encontró un pais...
            df.iloc[idx]['dstlat'] = tmp['LAT'].values[0]
            df.iloc[idx]['dstlong'] = tmp['LONG'].values[0]
            df.iloc[idx]['dstname'] = tmp['SHORT_NAME'].values[0]
        else: # Si no encuentra, el destino es un campus (Tabla Felipe)
            tmp = campusDevice[campusDevice['Device'] == df.iloc[idx]['device_name']]
            if len(tmp) == 0: # Si no encuentra, pondremos como MTY el default
                notFound.add(idx)
                print('dst MTY:',idx)   
                tmp = campusDevice[campusDevice['Device'] == 'CP-MTY-1'] 
            df.iloc[idx]['dstlat'] = tmp['latitud'].values[0]
            df.iloc[idx]['dstlong'] = tmp['longitud'].values[0]
            df.iloc[idx]['dstname'] = tmp['Nombre'].values[0]

    except Exception as e:
        notFound.add(idx)
        print('destiny:',idx,e)   

src MTY: 0
src MTY: 1
src MTY: 2
src MTY: 3
src MTY: 4
src MTY: 5
src MTY: 6
src MTY: 7
src MTY: 8
src MTY: 9
src MTY: 10
src MTY: 11
src MTY: 12
src MTY: 13
src MTY: 14
src MTY: 15
src MTY: 16
src MTY: 17
src MTY: 18
src MTY: 19
src MTY: 20
src MTY: 21
src MTY: 22
src MTY: 23
src MTY: 24
src MTY: 25
src MTY: 26
src MTY: 27
src MTY: 28
src MTY: 29
src MTY: 30
src MTY: 31
src MTY: 32
src MTY: 33
src MTY: 34
src MTY: 35
src MTY: 36
src MTY: 37
src MTY: 38
src MTY: 39
src MTY: 40
src MTY: 41
src MTY: 42
src MTY: 43
src MTY: 44
src MTY: 45
src MTY: 46
src MTY: 47
src MTY: 48
src MTY: 49
src MTY: 50
src MTY: 51
src MTY: 52
src MTY: 53
src MTY: 54
src MTY: 55
src MTY: 56
src MTY: 57
src MTY: 58
src MTY: 59
src MTY: 60
src MTY: 61
src MTY: 62
src MTY: 63
src MTY: 64
src MTY: 65
src MTY: 66
src MTY: 67
src MTY: 68
src MTY: 69
src MTY: 70
src MTY: 71
src MTY: 72
src MTY: 73
src MTY: 74
src MTY: 75
src MTY: 76
src MTY: 78
src MTY: 80
src MTY: 81
src MTY: 82
src MTY: 83
src MTY: 84
src MTY: 85
sr

src MTY: 666
src MTY: 667
src MTY: 668
src MTY: 669
src MTY: 670
src MTY: 671
src MTY: 672
src MTY: 673
src MTY: 674
src MTY: 675
src MTY: 676
src MTY: 677
src MTY: 678
src MTY: 679
src MTY: 680
src MTY: 681
src MTY: 682
src MTY: 684
src MTY: 685
src MTY: 686
src MTY: 687
src MTY: 688
src MTY: 689
src MTY: 690
src MTY: 691
src MTY: 692
src MTY: 693
src MTY: 694
src MTY: 695
src MTY: 696
src MTY: 697
src MTY: 698
src MTY: 699
src MTY: 700
src MTY: 701
src MTY: 702
src MTY: 703
src MTY: 704
src MTY: 705
src MTY: 706
src MTY: 707
src MTY: 708
src MTY: 709
src MTY: 710
src MTY: 711
src MTY: 712
src MTY: 713
src MTY: 714
src MTY: 715
src MTY: 716
src MTY: 717
src MTY: 718
src MTY: 719
src MTY: 720
src MTY: 721
src MTY: 722
src MTY: 723
src MTY: 724
src MTY: 725
src MTY: 726
src MTY: 727
src MTY: 728
src MTY: 729
src MTY: 730
src MTY: 731
src MTY: 732
src MTY: 733
src MTY: 734
src MTY: 735
src MTY: 736
src MTY: 737
src MTY: 738
src MTY: 739
src MTY: 740
src MTY: 741
src MTY: 742
src MTY: 743

src MTY: 1315
src MTY: 1316
src MTY: 1317
src MTY: 1318
src MTY: 1319
src MTY: 1320
src MTY: 1321
src MTY: 1322
src MTY: 1323
src MTY: 1324
src MTY: 1325
src MTY: 1326
src MTY: 1327
src MTY: 1328
src MTY: 1329
src MTY: 1330
src MTY: 1331
src MTY: 1332
src MTY: 1333
src MTY: 1334
src MTY: 1335
src MTY: 1336
src MTY: 1337
src MTY: 1338
src MTY: 1339
src MTY: 1340
src MTY: 1341
src MTY: 1342
src MTY: 1343
src MTY: 1344
src MTY: 1345
src MTY: 1346
src MTY: 1347
src MTY: 1348
src MTY: 1349
src MTY: 1350
src MTY: 1351
src MTY: 1352
src MTY: 1353
src MTY: 1354
src MTY: 1355
src MTY: 1356
src MTY: 1357
src MTY: 1358
src MTY: 1359
src MTY: 1360
src MTY: 1361
src MTY: 1362
src MTY: 1363
src MTY: 1364
src MTY: 1365
src MTY: 1366
src MTY: 1367
src MTY: 1368
src MTY: 1369
src MTY: 1370
src MTY: 1371
src MTY: 1372
src MTY: 1373
src MTY: 1374
src MTY: 1375
src MTY: 1376
src MTY: 1377
src MTY: 1378
src MTY: 1379
src MTY: 1380
src MTY: 1381
src MTY: 1382
src MTY: 1383
src MTY: 1384
src MTY: 1385
src MT

src MTY: 1917
src MTY: 1918
src MTY: 1919
src MTY: 1920
src MTY: 1921
src MTY: 1922
src MTY: 1923
src MTY: 1924
src MTY: 1925
src MTY: 1926
src MTY: 1927
src MTY: 1928
src MTY: 1929
src MTY: 1930
src MTY: 1931
src MTY: 1932
src MTY: 1933
src MTY: 1934
src MTY: 1935
src MTY: 1936
src MTY: 1937
src MTY: 1938
src MTY: 1939
src MTY: 1940
src MTY: 1941
src MTY: 1942
src MTY: 1943
src MTY: 1944
src MTY: 1945
src MTY: 1946
src MTY: 1947
src MTY: 1948
src MTY: 1949
src MTY: 1950
src MTY: 1951
src MTY: 1952
src MTY: 1953
src MTY: 1954
src MTY: 1955
src MTY: 1956
src MTY: 1957
src MTY: 1958
src MTY: 1959
src MTY: 1960
src MTY: 1961
src MTY: 1962
src MTY: 1963
src MTY: 1964
src MTY: 1965
src MTY: 1966
src MTY: 1967
src MTY: 1968
src MTY: 1969
src MTY: 1970
src MTY: 1971
src MTY: 1972
src MTY: 1973
src MTY: 1974
src MTY: 1975
src MTY: 1976
src MTY: 1977
src MTY: 1978
src MTY: 1979
src MTY: 1980
src MTY: 1981
src MTY: 1982
src MTY: 1983
src MTY: 1984
src MTY: 1985
src MTY: 1986
src MTY: 1987
src MT

src MTY: 2538
src MTY: 2539
src MTY: 2540
src MTY: 2541
src MTY: 2542
src MTY: 2543
src MTY: 2544
src MTY: 2545
src MTY: 2546
src MTY: 2547
src MTY: 2548
src MTY: 2549
src MTY: 2550
src MTY: 2551
src MTY: 2552
src MTY: 2553
src MTY: 2554
src MTY: 2555
src MTY: 2556
src MTY: 2557
src MTY: 2558
src MTY: 2559
src MTY: 2560
src MTY: 2561
src MTY: 2562
src MTY: 2563
src MTY: 2564
src MTY: 2565
src MTY: 2566
src MTY: 2567
src MTY: 2568
src MTY: 2569
src MTY: 2570
src MTY: 2571
src MTY: 2572
src MTY: 2573
src MTY: 2574
src MTY: 2575
src MTY: 2576
src MTY: 2577
src MTY: 2578
src MTY: 2579
src MTY: 2580
src MTY: 2581
src MTY: 2582
src MTY: 2583
src MTY: 2584
src MTY: 2585
src MTY: 2586
src MTY: 2587
src MTY: 2588
src MTY: 2589
src MTY: 2590
src MTY: 2591
src MTY: 2592
src MTY: 2593
src MTY: 2594
src MTY: 2595
src MTY: 2596
src MTY: 2597
src MTY: 2598
src MTY: 2599
src MTY: 2600
src MTY: 2601
src MTY: 2602
src MTY: 2603
src MTY: 2604
src MTY: 2605
src MTY: 2606
src MTY: 2607
src MTY: 2608
src MT

src MTY: 3175
src MTY: 3176
src MTY: 3177
src MTY: 3178
src MTY: 3179
src MTY: 3180
src MTY: 3181
src MTY: 3182
src MTY: 3183
src MTY: 3184
src MTY: 3185
src MTY: 3186
src MTY: 3187
src MTY: 3188
src MTY: 3189
src MTY: 3190
src MTY: 3191
src MTY: 3192
src MTY: 3193
src MTY: 3194
src MTY: 3195
src MTY: 3196
src MTY: 3197
src MTY: 3198
src MTY: 3199
src MTY: 3200
src MTY: 3201
src MTY: 3202
src MTY: 3203
src MTY: 3204
src MTY: 3205
src MTY: 3206
src MTY: 3207
src MTY: 3208
src MTY: 3209
src MTY: 3210
src MTY: 3211
src MTY: 3212
src MTY: 3213
src MTY: 3214
src MTY: 3215
src MTY: 3216
src MTY: 3217
src MTY: 3218
src MTY: 3219
src MTY: 3220
src MTY: 3221
src MTY: 3222
src MTY: 3223
src MTY: 3224
src MTY: 3225
src MTY: 3226
src MTY: 3227
src MTY: 3228
src MTY: 3229
src MTY: 3230
src MTY: 3231
src MTY: 3232
src MTY: 3233
src MTY: 3234
src MTY: 3235
src MTY: 3236
src MTY: 3237
src MTY: 3238
src MTY: 3239
src MTY: 3240
src MTY: 3241
src MTY: 3242
src MTY: 3243
src MTY: 3244
src MTY: 3246
src MT

src MTY: 3791
src MTY: 3792
src MTY: 3793
src MTY: 3794
src MTY: 3795
src MTY: 3796
src MTY: 3797
src MTY: 3798
src MTY: 3799
src MTY: 3800
src MTY: 3801
src MTY: 3802
src MTY: 3803
src MTY: 3804
src MTY: 3805
src MTY: 3806
src MTY: 3807
src MTY: 3808
src MTY: 3809
src MTY: 3810
src MTY: 3811
src MTY: 3812
src MTY: 3813
src MTY: 3814
src MTY: 3815
src MTY: 3816
src MTY: 3817
src MTY: 3818
src MTY: 3819
src MTY: 3820
src MTY: 3821
src MTY: 3822
src MTY: 3823
src MTY: 3824
src MTY: 3825
src MTY: 3826
src MTY: 3827
src MTY: 3828
src MTY: 3829
src MTY: 3830
src MTY: 3831
src MTY: 3832
src MTY: 3833
src MTY: 3834
src MTY: 3835
src MTY: 3836
src MTY: 3837
src MTY: 3838
src MTY: 3839
src MTY: 3840
src MTY: 3841
src MTY: 3842
src MTY: 3843
src MTY: 3844
src MTY: 3845
src MTY: 3846
src MTY: 3847
src MTY: 3848
src MTY: 3849
src MTY: 3850
src MTY: 3851
src MTY: 3852
src MTY: 3853
src MTY: 3854
src MTY: 3855
src MTY: 3856
src MTY: 3857
src MTY: 3858
src MTY: 3859
src MTY: 3860
src MTY: 3861
src MT

src MTY: 4416
src MTY: 4417
src MTY: 4418
src MTY: 4419
src MTY: 4420
src MTY: 4421
src MTY: 4422
src MTY: 4423
src MTY: 4424
src MTY: 4425
src MTY: 4426
src MTY: 4427
src MTY: 4428
src MTY: 4429
src MTY: 4430
src MTY: 4431
src MTY: 4432
src MTY: 4433
src MTY: 4434
src MTY: 4435
src MTY: 4436
src MTY: 4437
src MTY: 4438
src MTY: 4439
src MTY: 4440
src MTY: 4441
src MTY: 4442
src MTY: 4443
src MTY: 4444
src MTY: 4445
src MTY: 4446
src MTY: 4447
src MTY: 4448
src MTY: 4449
src MTY: 4450
src MTY: 4451
src MTY: 4452
src MTY: 4453
src MTY: 4454
src MTY: 4455
src MTY: 4456
src MTY: 4457
src MTY: 4458
src MTY: 4459
src MTY: 4460
src MTY: 4461
src MTY: 4462
src MTY: 4463
src MTY: 4464
src MTY: 4465
src MTY: 4466
src MTY: 4467
src MTY: 4468
src MTY: 4469
src MTY: 4470
src MTY: 4471
src MTY: 4472
src MTY: 4473
src MTY: 4474
src MTY: 4475
src MTY: 4476
src MTY: 4477
src MTY: 4478
src MTY: 4479
src MTY: 4480
src MTY: 4481
src MTY: 4482
src MTY: 4483
src MTY: 4484
src MTY: 4485
src MTY: 4486
src MT

In [20]:
campusDevice[campusDevice['Device'] == 'CP-MTY-1'] 

,Device,Campus,latitud,longitud,Nombre
47,CP-MTY-1,MTY,25.65142,-100.290328,Monterrey


In [21]:
df.to_json(os.path.expanduser('~/NorsePi/XML/LastHour.json'),orient='index')

In [16]:
df[df['srcname'] == 'MTY']

,direction,device_name,time_generated,src,srcloc,dst,dstloc,subtype,threatid,severity,srcname,srclat,srclong,dstname,dstlat,dstlong,src_alpha2,dst_alpha2,color_ataque
0,client-to-server,UTM-CUM,2018/11/20 10:50:43,10.74.33.110,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",104.248.7.51,"{'@code': 'United States', '@cc': 'US', '#text...",vulnerability,Exim Dovecot Mail Address Handling Command Inj...,high,MTY,25.6514,-100.29,United States,39.8282,-98.5795,10.0.0.0-10.255.255.255,US,#f48154
1,client-to-server,UTM-CUM,2018/11/20 10:50:43,10.74.33.110,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",185.246.155.112,"{'@code': 'Netherlands', '@cc': 'NL', '#text':...",vulnerability,Exim Dovecot Mail Address Handling Command Inj...,high,MTY,25.6514,-100.29,Netherlands,52.5,5.75,10.0.0.0-10.255.255.255,NL,#f48154
2,client-to-server,UTM-CUM,2018/11/20 10:19:06,10.74.46.247,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",156.154.128.70,"{'@code': 'United States', '@cc': 'US', '#text...",spyware,Suspicious DNS Query (generic:earkdjplpl.us),medium,MTY,25.6514,-100.29,United States,39.8282,-98.5795,10.0.0.0-10.255.255.255,US,#d9ff7f
3,client-to-server,UTM-CUM,2018/11/20 10:08:54,10.74.46.247,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",192.5.6.30,"{'@code': 'United States', '@cc': 'US', '#text...",spyware,generic:c.o5o4o6.com,medium,MTY,25.6514,-100.29,United States,39.8282,-98.5795,10.0.0.0-10.255.255.255,US,#d9ff7f
4,client-to-server,UTM-CUM,2018/11/20 09:53:09,10.74.46.247,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",212.18.248.42,"{'@code': 'United Kingdom', '@cc': 'GB', '#tex...",spyware,Suspicious DNS Query (generic:tracker.blackuni...,medium,MTY,25.6514,-100.29,Gabon,-1,11.75,10.0.0.0-10.255.255.255,GB,#d9ff7f
5,client-to-server,CP-MRL-2,2018/11/20 09:44:45,10.26.2.9,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",207.248.224.71,"{'@code': 'Mexico', '@cc': 'MX', '#text': 'Mex...",spyware,generic:eluxer.net,medium,MTY,25.6514,-100.29,Mexico,23,-102,10.0.0.0-10.255.255.255,MX,#d9ff7f
6,client-to-server,CP-MRL-2,2018/11/20 09:41:31,10.26.2.9,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",192.55.83.30,"{'@code': 'United States', '@cc': 'US', '#text...",spyware,generic:eluxer.net,medium,MTY,25.6514,-100.29,United States,39.8282,-98.5795,10.0.0.0-10.255.255.255,US,#d9ff7f
7,client-to-server,CP-MRL-2,2018/11/20 09:41:30,10.26.2.9,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",207.248.224.71,"{'@code': 'Mexico', '@cc': 'MX', '#text': 'Mex...",spyware,generic:eluxer.net,medium,MTY,25.6514,-100.29,Mexico,23,-102,10.0.0.0-10.255.255.255,MX,#d9ff7f
8,client-to-server,CP-MRL-2,2018/11/20 09:41:30,10.26.2.9,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",192.228.79.201,"{'@code': 'United States', '@cc': 'US', '#text...",spyware,generic:eluxer.net,medium,MTY,25.6514,-100.29,United States,39.8282,-98.5795,10.0.0.0-10.255.255.255,US,#d9ff7f
9,client-to-server,CP-MRL-2,2018/11/20 09:41:30,10.26.2.9,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",199.7.91.13,"{'@code': 'United States', '@cc': 'US', '#text...",spyware,generic:eluxer.net,medium,MTY,25.6514,-100.29,United States,39.8282,-98.5795,10.0.0.0-10.255.255.255,US,#d9ff7f


In [14]:
df[df['dstname'] == 'Hospital San Jose']

,direction,device_name,time_generated,src,srcloc,dst,dstloc,subtype,threatid,severity,srcname,srclat,srclong,dstname,dstlat,dstlong,src_alpha2,dst_alpha2,color_ataque
515,client-to-server,TS-SANJOSE-P1,2018/11/20 09:04:52,10.88.230.5,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",10.88.14.21,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",spyware,generic:uytestion.info,medium,MTY,25.6514,-100.29,Hospital San Jose,25.6681,-100.351,10.0.0.0-10.255.255.255,10.0.0.0-10.255.255.255,#d9ff7f
569,server-to-client,TS-SANJOSE-P1,2018/11/20 09:03:05,91.92.120.134,"{'@code': 'Bulgaria', '@cc': 'BG', '#text': 'B...",10.88.164.36,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",spyware,CoinHive JavaScript Detection,medium,Bangladesh,24,90,Hospital San Jose,25.6681,-100.351,BG,10.0.0.0-10.255.255.255,#d9ff7f
1713,server-to-client,TS-SANJOSE-P1,2018/11/20 08:40:58,91.92.120.139,"{'@code': 'Bulgaria', '@cc': 'BG', '#text': 'B...",10.88.164.36,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",spyware,CoinHive JavaScript Detection,medium,Bangladesh,24,90,Hospital San Jose,25.6681,-100.351,BG,10.0.0.0-10.255.255.255,#d9ff7f
4458,client-to-server,TS-SANJOSE-P1,2018/11/20 08:25:17,10.88.230.5,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",10.88.14.21,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",spyware,generic:c.o5o4o6.com,medium,MTY,25.6514,-100.29,Hospital San Jose,25.6681,-100.351,10.0.0.0-10.255.255.255,10.0.0.0-10.255.255.255,#d9ff7f


In [9]:
df[df['severity']=='critical']

,direction,device_name,time_generated,src,srcloc,dst,dstloc,subtype,threatid,severity,srcname,srclat,srclong,dstname,dstlat,dstlong,src_alpha2,dst_alpha2,color_ataque
77,client-to-server,CP-MRL-2,2018/11/20 09:18:06,66.240.205.34,"{'@code': 'United States', '@cc': 'US', '#text...",10.26.0.98,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",spyware,ZeroAccess.Gen Command and Control Traffic,critical,United States,39.8282,-98.5795,Campus Morelia,19.6565,-101.164,US,10.0.0.0-10.255.255.255,#ff4660
1989,client-to-server,CP-LEO-1,2018/11/20 08:40:21,122.112.212.238,"{'@code': 'China', '@cc': 'CN', '#text': 'China'}",10.24.140.6,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",vulnerability,Microsoft IIS WebDAV ScStoragePathFromUrl Buff...,critical,China,35,105,León,21.1663,-101.716,CN,10.0.0.0-10.255.255.255,#ff4660
3125,client-to-server,CP-LEO-1,2018/11/20 08:32:01,66.240.205.34,"{'@code': 'United States', '@cc': 'US', '#text...",10.24.145.24,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",spyware,ZeroAccess.Gen Command and Control Traffic,critical,United States,39.8282,-98.5795,León,21.1663,-101.716,US,10.0.0.0-10.255.255.255,#ff4660
3644,client-to-server,CP-GDL-1,2018/11/20 08:29:22,66.240.205.34,"{'@code': 'United States', '@cc': 'US', '#text...",132.254.232.163,"{'@code': 'Mexico', '@cc': 'MX', '#text': 'Mex...",spyware,ZeroAccess.Gen Command and Control Traffic,critical,United States,39.8282,-98.5795,Mexico,23,-102,US,MX,#ff4660
4812,client-to-server,prod451azap01,2018/11/20 08:24:17,115.159.98.117,"{'@code': 'China', '@cc': 'CN', '#text': 'China'}",10.97.135.197,"{'@code': '10.0.0.0-10.255.255.255', '@cc': '1...",vulnerability,Microsoft IIS WebDAV ScStoragePathFromUrl Buff...,critical,China,35,105,Azure South Central US,26.041,-98.4198,CN,10.0.0.0-10.255.255.255,#ff4660


In [10]:
df.iloc[58]

direction                                          client-to-server
device_name                                                CP-CSF-1
time_generated                                  2018/11/20 09:19:39
src                                                     10.49.23.40
srcloc            {'@code': '10.0.0.0-10.255.255.255', '@cc': '1...
dst                                                  194.169.218.14
dstloc            {'@code': 'United Kingdom', '@cc': 'GB', '#tex...
subtype                                                     spyware
threatid                                       generic:battlenet.la
severity                                                     medium
srcname                                                         MTY
srclat                                                      25.6514
srclong                                                     -100.29
dstname                                                       Gabon
dstlat                                          